In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 

# Expectations

In [ ]:
#Import POS file, AMT, Pallet Quantity file, Sku Conversion Sheet from THD to MAS 500

#Use AMT sheet, merge in pallet counts per sku. Assign this a new variable "OOS_Report".

#To "OOS Report" merge in POS OH units, fill na with 0. 

#Create a new column, labelled "OH PLTs". This column is the rounded of (OH Units / Pallet counts)

#Check for orders, import the MAS 500 data. Merge the sku conversion to the file.
#Change the customer number to store number. 

#Combine the open with the closed list. 

#Merge the sum of the totals by store by sku with the "OOS_Report" file. 

#Create boolean filter to take away anything that total doesn't equal 0 pallets.

# Live Code

In [ ]:
#Files to import per run of code
POS_File = pd.read_excel(#Current Week's Updated Inventory, sheet_name=6)
AMT = pd.read_excel(#In-Store Assortment List)
Sku_Info = pd.read_excel(#Sku related file ie. pallet sizes per sku)

In [ ]:
POS_File.head(3)

In [ ]:
AMT.head(3)

In [ ]:
Sku_Info.head(3)

In [ ]:
#Removing unnecessary information
AMT.drop(columns = ['STR NAME', 'SET NAME'], inplace = True)

In [ ]:
#Reassigning file with filter of only active skus in each respective store
AMT = AMT[AMT['AMT'] == 'Y']

In [ ]:
#Merging together detailed sku information and the assortment and removing uncessary columns. Into new dataframe.
OOS_Report = AMT.merge(Sku_Info, how = 'left', left_on = 'Sku', right_on = 'ITM')
OOS_Report.drop(columns = ['Item', 'Description', 'ITM', 'ITM DESC'], inplace = True)

In [ ]:
#Keeping numeric values as integars rather than floats
OOS_Report['PLT'] = OOS_Report['PLT'].astype('int')

In [ ]:
OOS_Report.head()

In [ ]:
#Merging in the weekly updated POS and inventory to the assortment file
OOS_Report = OOS_Report.merge(POS_File, how = 'left', left_on = ['STR NBR', 'Sku'], right_on = ['Store', 'Item'])

In [ ]:
#Removing unnecessary columns from the merge
OOS_Report.drop(columns = ['BYO', 'Market', 'Store', 'Item', 'Item Description'], inplace = True)

In [ ]:
#Covering all "N/A" cells with 0s to avoid calculation errors
OOS_Report = OOS_Report.fillna(0)

In [ ]:
#Keeping numeric values as integars rather than floats
OOS_Report[2019] = OOS_Report[2019].astype('int')

In [ ]:
#Renaming columns for everyone viewing the file
OOS_Report.columns = ['BYO NBR', 'MKT NBR', 'STR NBR', 'Sku', 'Sku Description', 'AMT', 'PLT', 'OH Units']

In [ ]:
OOS_Report.head()

In [ ]:
#Calculating inventory pallets per sku per store for actual out of stock skus
OOS_Report['OH PLTs'] = (OOS_Report['OH Units'] / OOS_Report['PLT']).round().astype('int')

In [ ]:
OOS_Report.head()

Output 1: All OOS (Not counting on order)

In [ ]:
#First output report, all current out of stock items (not looking at what is open as an order)
True_OOS = OOS_Report[OOS_Report['OH PLTs'] == 0]

Continue: Adding In On-Orders

In [ ]:
MAS500 = pd.read_excel(#Adding in the dataframes for the open orders, sheet_name=None)

In [ ]:
#Combining the seperated dataframes into one
MAS500 = MAS500['Open'].append(MAS500['Closed'], ignore_index=True, sort='True')

In [ ]:
#Removing unnecessary columns from file
MAS500.drop(columns = ['Segment1', 'Segment1 Description', 'Segment2', 'Segment2 Description', 'Segment3', 'Segment3 Description'], inplace = True)

In [ ]:
MAS500.head()

In [ ]:
#Creating group by store and item for total units being ordered per sku per store
MAS500_Totals = MAS500.groupby(by = ['Cust', 'Item']).sum()

In [ ]:
#Removing any duplicated data from the output of the sum 
MAS500_Totals = MAS500_Totals.drop_duplicates(keep = 'first')

In [ ]:
#Removing unnecessary columns
MAS500_Totals.drop(columns = ['Count Per Pallet', 'Customer PO Number', 'Ext Amt', 'SO Nbr - Rel', 'Unit Price'], inplace = True)

In [ ]:
MAS500_Totals.head()

In [ ]:
#Resetting the index to get out of the multi-index dataframe
MAS500_Totals.reset_index(inplace = True)

In [ ]:
#Reassign the store numbers to match the inventory file from above
MAS500_Totals['Cust'] = [x - 6000000 for x in MAS500_Totals['Cust']]

In [ ]:
MAS500_Totals.tail()

In [ ]:
#Combining the order details with the sku information to align the different skus and removing unnecessary columns
MAS500_Totals = MAS500_Totals.merge(Sku_Info, how = 'left', on = 'Item')
MAS500_Totals.drop(columns = ['Description', 'ITM DESC', 'PLT'], inplace = True)

In [ ]:
#Removing any data that does not have a valid output
MAS500_Totals.dropna(how = 'any', inplace = True)

In [ ]:
#Turning the sku numbers to integars from floats
MAS500_Totals['ITM'] = MAS500_Totals['ITM'].astype('int')

In [ ]:
MAS500_Totals.head()

In [ ]:
OOS_Report.head()

In [ ]:
#Merging the original inventory file with the orders to see exactly what inventory issue is already being addressed
POS_With_OO = OOS_Report.merge(MAS500_Totals, how = 'left', left_on = ['STR NBR', 'Sku'], right_on = ['Cust', 'ITM']).fillna(0)
POS_With_OO.head()

In [ ]:
#Removing unnecessary columns
POS_With_OO.drop(columns = ['Cust', 'Item', 'ITM'], inplace = True)

In [ ]:
#Adding in a new pallet count by adding store inventory with order units per store per sku
POS_With_OO['OH & OO PLT'] = POS_With_OO['OH PLTs'] + POS_With_OO['Pallet Ordered']

In [ ]:
POS_With_OO.head()

Output 2: OOS without an Order

In [ ]:
#New file that shows inventory issues that have not been addressed
Actual_OO = POS_With_OO[POS_With_OO['OH & OO PLT'] == 0]

Exporting Results

In [ ]:
#Exporting both files out to excel
True_OOS.to_excel('All OOS skus.xlsx', index = False)
Actual_OO.to_excel('OOS without Orders.xlsx', index = False)